In [1]:
import numpy as np
import pandas as pd
import re
import math
import itertools
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import collections 

from ast import literal_eval
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import namedtuple
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
from  scipy import spatial
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [2]:
d2v_model = Doc2Vec.load('../model/word_embedding/Doc2vec_new_small2_4.model')

# modeling

### 데이터셋 불러오기

In [3]:
final_list  = pd.read_excel('./new_data/final_list_score8.xlsx')
inv_list  = pd.read_excel('./new_data/invest_df.xlsx')
final_list.fillna('nan', inplace=True)
final_list['token'] = final_list.token.apply(lambda x:literal_eval(x))

In [4]:
final_list['invest'] = [str(x).split(',') for x in final_list['invest']]

In [5]:
invest = list(set([el.strip() for el in final_list['invest'].sum()]))
inv_list.set_index('invest', inplace=True)
inv_list = inv_list.apply(minmax_scale)

In [6]:
final_list.shape

(83, 9)

In [7]:
inv_list = inv_list.where(inv_list > np.quantile(inv_list['invest_num'], 0.25), np.quantile(inv_list['invest_num'], 0.25))

### Document vector

In [8]:
paragraph_id = {k:v for k, v in enumerate(d2v_model.docvecs.doctags.keys())}
doc_vec = {paragraph_id[k]:list(v) for k,v in enumerate(d2v_model.docvecs.vectors_docs)}

In [9]:
inv_tech_df = pd.DataFrame(columns = ['investor', 'tech', 'weights', 'company'])
for values_list in  final_list[['company', 'predicted_small_label1', 'predicted_small_label2', 'invest']].values:
#     print(values_list)
    for com in values_list[-1]:
#         print(com)
        if com.strip() in ['nan', '비공개', '크라우드펀딩', '사모펀드']:
            continue
        else:
            try:
                weights = inv_list.loc[com.strip()].values[0]
                inv_tech_df = inv_tech_df.append({'investor':com.strip(), 'tech':values_list[1], 'weights':weights, 'company':values_list[0]}, \
                                                 ignore_index=True)
            except KeyError:
                print(com)
                continue

사이버에이전트벤처즈 코리아
 PIA
 Mission Capital
 GC녹십자Genome
 스파크랩
 데일리파트너스
 원익
옐로금융그룹
 KB인베스트먼트
 한화인베스트먼트
옐로우독
 콜라보레이티브펀드
 알토스벤처스
현대기술투자
 비씨카드(BC카드)
 UTC인베스트먼트
 스마일게이트인베스트먼트
 미래에셋벤처투자
 KT인베스트먼트
 미래에셋벤처투자
카이트창업가재단
킹슬리 벤처스
 GS홈쇼핑
 DS앤파트너스
 뮤렉스파트너스
 아이디지캐피탈파트너스코리아
 케이비인베스트먼트
 아이디벤처스
 현대자동차 제로원엑셀러레이터
케이비인베스트먼트
에이티넘인베스트먼트
 신용보증기금 경기스타트업지점
 헤시드


In [10]:
inv = list(inv_list.index)
inv_idx = ["i%s" % str(i).zfill(3) for i in range(1,len(inv_list.index)+1)]
inv_node = pd.DataFrame({'node':inv,'node_idx':inv_idx})

In [11]:
inv_tech_df.rename(columns={'investor':'node'}, inplace=True)

In [12]:
inv_tech_df.to_excel('./last2/investor_graph.xlsx', index=False, encoding='cp949')

### 네트워크

In [13]:
def network_score(G) :
    tt =  { "degree":dict(G.degree()), 
            "degree_centrality":nx.degree_centrality(G), #normalized degree, degree가 클수록 높음
            "closeness_centrality":nx.closeness_centrality(G), 
            "betweenness_centrality":nx.betweenness_centrality(G), 
            "pagerank":nx.pagerank(G)}
    tt_df = pd.DataFrame(tt)
    tt_df['node'] = list(tt_df.index)
    tt_df = pd.merge(tt_df,node_df,how='left')
    return tt_df

In [14]:
small_label = list(set(final_list.predicted_small_label1.tolist() + final_list.predicted_small_label2.tolist()))
small_label = [label for label in small_label if label != 'nan']
comb_dv = list(itertools.combinations(small_label,2))

In [15]:
%%time
tech_similarity_list = list()
tech_mat = pd.DataFrame(index=small_label, columns=small_label)
for comb1, comb2 in comb_dv:
    cos_sim = cosine_similarity([doc_vec[comb1]], [doc_vec[comb2]])[0][0]
    tech_mat.loc[comb1,comb2] = cos_sim
    tech_mat.loc[comb2,comb1] = cos_sim
    tech_similarity_list.append(cos_sim)
tech_mat2 = tech_mat.where(tech_mat > 0.13, 0.0)
tech_mat3 = tech_mat2.where(tech_mat2 == 0, 1)

Wall time: 453 ms


In [16]:
t_edge_df = pd.DataFrame(columns= ['source', 'target', 'weights'])
for idx, comb in enumerate(comb_dv):
    edge =tech_mat3.loc[comb[0], comb[1]]
    if edge == 0.0:
        continue
    else:
        weight = tech_mat2.loc[comb[0], comb[1]]
        t_edge_df = t_edge_df.append({'source':comb[0], 'target':comb[1], 'weights':weight}, ignore_index=True)
t_edge = [tuple(element) for element in t_edge_df.values]

In [17]:
t_edge

[('생체유래 물질 분석 시스템', 'e-커머스 플랫폼', 0.14609941840171814),
 ('생체유래 물질 분석 시스템', '분자진단', 0.19310154020786285),
 ('생체신호 측정진단기기', '인지과학 SW', 0.15659192204475403),
 ('동영상 미디어 플랫폼', '영상콘텐츠', 0.15523482859134674),
 ('인간-인공지능 협업 시스템', '자산관리 서비스', 0.2407730370759964),
 ('인간-인공지능 협업 시스템', 'SNS 플랫폼', 0.14065206050872803),
 ('개인 맞춤형 건강관리 기기 및 플랫폼', '스마트 헬스 데이터 분석', 0.13513311743736267),
 ('크라우드펀딩', '빅데이터 기반 SW', 0.13307346403598785),
 ('크라우드펀딩', '건강정보케어 서비스', 0.13781927525997162),
 ('데이터 3D 변환 시각화 도구', '빅데이터 분석 및 시각화 플랫폼', 0.1488972306251526),
 ('면역화학진단', '분자진단', 0.21076862514019012),
 ('면역화학진단', 'Robotic Process Automation', 0.14846941828727722),
 ('면역화학진단', '음성인식 시스템', 0.1301812380552292),
 ('모바일 핀테크 보안', '인지과학 SW', 0.1420866847038269),
 ('모바일 핀테크 보안', '건강정보케어 서비스', 0.1692826747894287),
 ('지능형 O2O 서비스 데이터 분석 시스템', '외래관광객용 관광 O2O 서비스', 0.14467081427574158),
 ('실시간 건강관리 시스템', '건강정보케어 서비스', 0.44034552574157715),
 ('게임 플랫폼', '국내외 간편송금 및 결제', 0.13850168883800507),
 ('인지과학 SW', '건강정보케어 서비스', 0.15813273191

In [18]:
%%time
company = list(final_list['company'])
company_idx = ["c%s" % str(i).zfill(2) for i in range(1,len(company)+1)]
node_df = pd.DataFrame({'node':company,'node_idx':company_idx, 'token':final_list.token.tolist()})
sim_sentence = pd.DataFrame(columns=company, index=company)
comb_c = list(itertools.combinations(node_df.node.tolist(),2))
similarity_list = list()
for idx,comb in enumerate(comb_c):
    if idx % 100 == 0:
        print(comb[0], comb[1])
    sent1 = node_df.loc[node_df.node == comb[0], 'token'].values[0]
    sent2 = node_df.loc[node_df.node == comb[1], 'token'].values[0]
    sent1_vector = d2v_model.infer_vector(sent1)
    sent2_vector = d2v_model.infer_vector(sent2)
    sim_sentence.loc[comb[0], comb[1]] = cosine_similarity([sent1_vector], [sent2_vector])[0][0]
    sim_sentence.loc[comb[1], comb[0]] = cosine_similarity([sent2_vector], [sent1_vector])[0][0]
    similarity_list.append(cosine_similarity([sent1_vector], [sent2_vector])[0][0])
sim_sentence2 = sim_sentence.where(sim_sentence > np.quantile(similarity_list, 0.90), 0.0)
sim_sentence3 = sim_sentence2.where(sim_sentence2 == 0, 1)

(주)위즈벤처스 스마트토이
스마트토이 베이글랩스
트래블플랜 먼치팩토리
제노플랜 뷰티패스
렌딧 펀다
펀다 굳센컴퍼니
텍스트팩토리 쉐어트리츠
매스프레소 알고리마
클래스101 서큘러스
트위니 마스터지지
데이터앤애널리틱스 캐치잇플레이
밍글 하우투비즈랩
스켈터랩스 콴텍
티엔디엔 머지홀딩스
액션파워 바오바브코리아
베이글랩스 바이트
휴먼스케이프 리턴박스
뉴로핏 바오바브코리아
콴텍 (주)블루웨일컴퍼니
에임 마스터지지
올핀 하우투비즈랩
머니브레인 쉐어트리츠
바오바브코리아 루티헬스
플레인베이글 뷰티패스
마켓잇 페이플
먼치팩토리 ERGO
알고리즘랩스 머지홀딩스
아모랩 브이픽스메디칼
닥터다이어리 비링크헬스케어
리턴박스 (주)블루웨일컴퍼니
메디픽셀 하우투비즈랩
딥네츄럴 베드라디오
스터디짐 페이플
소셜클럽 바이트
베드라디오 알고리마
Wall time: 48.1 s


In [19]:
sim_sentence3

,(주)위즈벤처스,스마트토이,트래블플랜,제노플랜,어니스트펀드,렌딧,펀다,텍스트팩토리,매스프레소,주식회사 날비컴퍼니,...,슈퍼브에이아이,패스트비코리아,비링크헬스케어,(주)블루웨일컴퍼니,주식회사 셰어킴,아이팩토리,보이스루,베드라디오,알고리마,NICE비즈니스플랫폼
(주)위즈벤처스,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
스마트토이,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
트래블플랜,0,0,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
제노플랜,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
어니스트펀드,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
아이팩토리,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
보이스루,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
베드라디오,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
알고리마,0,0,1,0,1,0,0,0,1,0,...,1,0,1,0,0,0,1,0,0,0


In [20]:
edge_df = pd.DataFrame(columns= ['source', 'target', 'weights'])
for idx,comb in enumerate(comb_c):
    edge =sim_sentence3.loc[comb[0], comb[1]]
    if edge == 0.0:
        continue
    else:
        weight = sim_sentence2.loc[comb[0], comb[1]]
        edge_df = edge_df.append({'source':comb[0], 'target':comb[1], 'weights':weight}, ignore_index=True)
c_edge = [tuple(element) for element in edge_df.values]

In [21]:
small = list(set(list(final_list['predicted_small_label1']) + list(final_list['predicted_small_label2'])))
small_idx = ["s%s" % str(i).zfill(2) for i in range(1,len(small)+1)]
small_node = pd.DataFrame({'node':small,'node_idx':small_idx})
node_df = node_df.append(pd.DataFrame(data=small_node),ignore_index=True)

In [22]:
s_edge = []
for i in range(len(final_list)):
    company = final_list['company'][i]
    small1 = final_list['predicted_small_label1'][i]
    if small1 == 'nan':
        print(company)
    else:
        s_edge.append((small1, company,1))
    small2 = final_list['predicted_small_label2'][i]
    if small2 == 'nan':
        print(company)
    else:
        s_edge.append((small2, company,1))

리턴박스
쉐어트리츠
스터디짐
이콜트리
바이트
아이팩토리


In [39]:
company_tech = final_list[['company','predicted_label1','predicted_label2', 'predicted_small_label1', 'predicted_small_label2']]
company_tech.rename(columns={'company':'node'}, inplace=True)

In [40]:
node_df = node_df.append(pd.DataFrame(data=inv_node),ignore_index=True)
node_df.to_excel('./last2/new_node_df.xlsx', index=False, encoding='cp949')

In [41]:
i_edge = [(el[0], el[3],float(el[2])) for el in inv_tech_df.values]

In [42]:
total_edge = c_edge+s_edge+i_edge

In [43]:
network_df = pd.DataFrame(total_edge, columns=['Source', 'Target', 'Weight'])

In [44]:
network_df.to_csv('./last2/gephi/gephi_edge.csv', index=False, encoding='utf-8')

In [45]:
network_df.isnull().sum()

Source    0
Target    0
Weight    0
dtype: int64

In [46]:
total_edge2 = c_edge+s_edge+i_edge+t_edge

In [47]:
network_df2 = pd.DataFrame(total_edge2, columns=['Source', 'Target', 'Weight'])

In [48]:
network_df2.to_csv('./last2/gephi/gephi_edge(tech_connection).csv', index=False, encoding='utf-8')

### Graph(T-C-I)

In [49]:
nondirect_G = nx.Graph()
nondirect_G.add_weighted_edges_from(c_edge+s_edge+i_edge)
nondirect_score = network_score(nondirect_G)
nondirect_score.to_excel('./last2/non_tech_connection/network_score.xlsx', index=False, encoding='cp949')

In [50]:
searchfor = ['c', 'i']
dict_tech_com = nondirect_score.loc[nondirect_score.node_idx.str.contains('|'.join(searchfor)), ['node', 'node_idx']].sort_values('node_idx')
dict_tc = {k:v for k, v in zip(dict_tech_com['node'], dict_tech_com['node_idx'])}
dict_tc2 = {v:k for k, v in zip(dict_tech_com['node'], dict_tech_com['node_idx'])}

In [51]:
ns_df = nondirect_score[['node','node_idx', 'pagerank', 'closeness_centrality', 'betweenness_centrality']].sort_values('node_idx')
ns_df['Weight'] = ns_df[['pagerank', 'closeness_centrality', 'betweenness_centrality']].sum(axis=1)
ns_df = ns_df[['node','node_idx', 'Weight']].reset_index(drop=True)

In [52]:
def class_allocation(x):
    if x[0] == 'c':
        return 'company'
    elif x[0] == 's':
        return 'technology'
    elif x[0] == 'i':
        return 'investor'

In [53]:
ns_df['Attribute2'] = ns_df['node_idx'].apply(lambda x: class_allocation(x))
ns_df.columns = ['ID', 'Attribute1', 'Weight', 'Attribute2']
network_df.replace(dict_tc, inplace=True)
network_df.to_csv('./last2/gephi/gephi_edge2.csv', index=False, encoding='utf-8')
ns_df[['ID', 'Attribute1', 'Attribute2', 'Weight']].to_csv('./last2/gephi/gephi_node.csv', index=False, encoding='utf-8')

In [54]:
ns_df['ID'] = ns_df.ID.replace(dict_tc)
ns_df['Attribute1'] = ns_df.Attribute1.replace(dict_tc2)
ns_df[['ID', 'Attribute1', 'Attribute2', 'Weight']].to_csv('./last2/gephi/gephi_node2.csv', index=False, encoding='utf-8')

#### NTBF

In [55]:
company_network_result = nondirect_score[nondirect_score['node_idx'].isin(company_idx)].sort_values(["pagerank"], ascending=[False])
company_network_result['total_weight'] = company_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
company_network_result = company_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
company_network_result = pd.merge(company_network_result, company_tech)
company_network_result.to_excel('./last2/non_tech_connection/company_network_result.xlsx', index=False, encoding='cp949')

In [56]:
com_score1 = company_network_result.groupby('predicted_small_label1').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [57]:
com_score2 = company_network_result.groupby('predicted_small_label2').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [58]:
com_score2.rename(columns ={'total_weight':'total_weight2'}, inplace=True)

In [59]:
total_index = list(set(com_score1.index.tolist() + com_score2.index.tolist()))

In [60]:
company_total_df = pd.DataFrame(index=total_index)

In [61]:
com_score1.index.name = None
com_score2.index.name = None

In [62]:
company_total_df = company_total_df.join(com_score1)
company_total_df = company_total_df.join(com_score2)

In [63]:
company_total_df.fillna(0, inplace=True)

In [64]:
company_score = pd.DataFrame(company_total_df.mean(axis=1), columns=['total_weight']).sort_values('total_weight', ascending=False)

In [65]:
company_score.to_excel('./last2/non_tech_connection/company_based_score.xlsx')

#### Emerging Technology

In [66]:
tech_network_result = nondirect_score[nondirect_score['node_idx'].isin(small_idx)].sort_values(["pagerank"], ascending=[False])
tech_network_result['total_weight'] = tech_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
tech_network_result = tech_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
tech_network_result.to_excel('./last2/non_tech_connection/tech_network_result.xlsx', index=False, encoding='cp949')

In [67]:
tech_network_result[['node', 'total_weight']].sort_values('total_weight', ascending=False).to_excel('./last2/non_tech_connection/emerging_tech.xlsx', index=False)

In [68]:
tech_score = tech_network_result[['node', 'total_weight']].sort_values('total_weight', ascending=False).set_index('node')

In [69]:
tech_score.index.name=None

#### investor

In [70]:
inv_network_result = nondirect_score[nondirect_score['node_idx'].isin(inv_idx)].sort_values(["pagerank"], ascending=[False])
inv_network_result['total_weight'] = inv_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
inv_network_result = inv_network_result.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
inv_network_result = pd.merge(inv_network_result, inv_tech_df)
inv_network_result.to_excel('./last2/non_tech_connection/inv_network_result.xlsx', index=False, encoding='cp949')

In [71]:
inv_score = inv_network_result.groupby('tech').mean()[['total_weight']]

In [72]:
inv_score.sort_values('total_weight', ascending=False).to_excel('./last2/non_tech_connection/inv_tech.xlsx')

In [73]:
inv_score.index.name=None

#### Total weight

In [74]:
total_weight_index = list(set(company_score.index.tolist() + inv_score.index.tolist() + tech_network_result.node.tolist()))
total_score = pd.DataFrame(index=total_weight_index)

In [75]:
company_score.rename(columns={'total_weight':'NS'}, inplace=True)
tech_score.rename(columns={'total_weight':'TS'}, inplace=True)
inv_score.rename(columns={'total_weight':'IS'}, inplace=True)

In [76]:
company_score.to_excel('./last2/non_tech_connection/NS.xlsx')
tech_score.to_excel('./last2/non_tech_connection/TS.xlsx')
inv_score.to_excel('./last2/non_tech_connection/IS.xlsx')

In [77]:
total_score = total_score.join(company_score)
total_score = total_score.join(tech_score)
total_score = total_score.join(inv_score)

In [78]:
total_score_df = pd.DataFrame(total_score.fillna(0).sum(axis=1), columns=['Total score']).sort_values('Total score', ascending=False)

In [79]:
total_score_df.to_excel('./last2/non_tech_connection/TOS.xlsx')

### Graph(T-C-I) - tech_connection

In [80]:
nondirect_G2 = nx.Graph()
nondirect_G2.add_weighted_edges_from(c_edge+s_edge+i_edge+t_edge)
nondirect_score2 = network_score(nondirect_G2)
nondirect_score2.to_excel('./last2/tech_connection/network_score.xlsx', index=False, encoding='cp949')

In [81]:
nondirect_score2

,degree,degree_centrality,closeness_centrality,betweenness_centrality,pagerank,node,node_idx,token
0,10,0.047170,0.366149,0.013702,0.006245,(주)위즈벤처스,c01,"[빅데이터, sns, 솔루션, 분석, 실시간, 소개, 본사, 사업목표, sns, 빅..."
1,17,0.080189,0.384755,0.023576,0.008708,데이터앤애널리틱스,c14,"[서비스, 웹서비스, 스타트업, 벤처캐피탈, 인공지능, ai, 머신러닝, 데이터분석..."
2,16,0.075472,0.375887,0.028251,0.008910,티엔디엔,c18,"[티엔디엔, 토스, 결제서비스, 여행, 서비스, 웹서비스, 마케팅, 소프트웨어, 플..."
3,25,0.117925,0.428283,0.082150,0.012675,언더폭스,c33,"[1인, 미디어, 후원, 플랫폼, 크리에이터, 소비, 비용, 부담, 시청자들, 후원..."
4,14,0.066038,0.377897,0.020424,0.008346,딥네츄럴,c58,"[대화, 인공지능, 데이터, 생산, 스타트업, 영화, 드라마, 등장, 주인공, 등장..."
...,...,...,...,...,...,...,...,...
303,1,0.004717,0.269036,0.000000,0.000780,기술보증기금,i036,NaN
304,1,0.004717,0.288043,0.000000,0.000819,더인벤션랩,i035,NaN
305,1,0.004717,0.288043,0.000000,0.000819,더인벤션랩,i035,NaN
306,1,0.004717,0.235818,0.000000,0.000730,제주창조경제혁신센터,i109,NaN


In [82]:
searchfor = ['c', 'i']
dict_tech_com2 = nondirect_score2.loc[nondirect_score2.node_idx.str.contains('|'.join(searchfor)), ['node', 'node_idx']].sort_values('node_idx')
dict_tc_1 = {k:v for k, v in zip(dict_tech_com2['node'], dict_tech_com2['node_idx'])}
dict_tc2_1 = {v:k for k, v in zip(dict_tech_com2['node'], dict_tech_com2['node_idx'])}

In [83]:
ns_df2 = nondirect_score2[['node','node_idx', 'pagerank', 'closeness_centrality', 'betweenness_centrality']].sort_values('node_idx')
ns_df2['Weight'] = ns_df2[['pagerank', 'closeness_centrality', 'betweenness_centrality']].sum(axis=1)
ns_df2 = ns_df2[['node','node_idx', 'Weight']].reset_index(drop=True)

In [84]:
def class_allocation(x):
    if x[0] == 'c':
        return 'company'
    elif x[0] == 's':
        return 'technology'
    elif x[0] == 'i':
        return 'investor'

In [85]:
ns_df2['Attribute2'] = ns_df2['node_idx'].apply(lambda x: class_allocation(x))
ns_df2.columns = ['ID', 'Attribute1', 'Weight', 'Attribute2']
network_df2.replace(dict_tc_1, inplace=True)
network_df2.to_csv('./last2/gephi/gephi_edge2(tech_connection).csv', index=False, encoding='utf-8')
ns_df2[['ID', 'Attribute1', 'Attribute2', 'Weight']].to_csv('./last2/gephi/gephi_node(tech_connection).csv', index=False, encoding='utf-8')

In [86]:
ns_df2['ID'] = ns_df.ID.replace(dict_tc_1)
ns_df2['Attribute1'] = ns_df.Attribute1.replace(dict_tc2_1)
ns_df2[['ID', 'Attribute1', 'Attribute2', 'Weight']].to_csv('./last2/gephi/gephi_node2(tech_connection).csv', index=False, encoding='utf-8')

#### NTBF

In [88]:
company_network_result2 = nondirect_score2[nondirect_score2['node_idx'].isin(company_idx)].sort_values(["pagerank"], ascending=[False])
company_network_result2['total_weight'] = company_network_result[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
company_network_result2 = company_network_result2.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
company_network_result2 = pd.merge(company_network_result2, company_tech)
company_network_result2.to_excel('./last2/tech_connection/company_network_result.xlsx', index=False, encoding='cp949')

In [89]:
com_score1 = company_network_result2.groupby('predicted_small_label1').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [90]:
com_score2 = company_network_result2.groupby('predicted_small_label2').mean()[['total_weight']].sort_values('total_weight', ascending=False)

In [91]:
com_score2.rename(columns ={'total_weight':'total_weight2'}, inplace=True)

In [92]:
total_index = list(set(com_score1.index.tolist() + com_score2.index.tolist()))

In [93]:
company_total_df2 = pd.DataFrame(index=total_index)

In [94]:
com_score1.index.name = None
com_score2.index.name = None

In [97]:
company_total_df2 = company_total_df2.join(com_score1)
company_total_df2 = company_total_df2.join(com_score2)

In [98]:
company_total_df2.fillna(0, inplace=True)

In [102]:
company_score2 = pd.DataFrame(company_total_df2.mean(axis=1), columns=['total_weight']).sort_values('total_weight', ascending=False)

In [103]:
company_score2.to_excel('./last2/tech_connection/company_based_score.xlsx')

#### Emerging Technology

In [104]:
tech_network_result2 = nondirect_score2[nondirect_score2['node_idx'].isin(small_idx)].sort_values(["pagerank"], ascending=[False])
tech_network_result2['total_weight'] = tech_network_result2[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
tech_network_result2 = tech_network_result2.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
tech_network_result2.to_excel('./last2/tech_connection/tech_network_result.xlsx', index=False, encoding='cp949')

In [105]:
tech_network_result2[['node', 'total_weight']].sort_values('total_weight', ascending=False).to_excel('./last2/tech_connection/emerging_tech.xlsx', index=False)

In [106]:
tech_score2 = tech_network_result2[['node', 'total_weight']].sort_values('total_weight', ascending=False).set_index('node')

In [107]:
tech_score2.index.name=None

#### investor

In [108]:
inv_network_result2 = nondirect_score2[nondirect_score2['node_idx'].isin(inv_idx)].sort_values(["pagerank"], ascending=[False])
inv_network_result2['total_weight'] = inv_network_result2[['closeness_centrality','betweenness_centrality', 'pagerank']].sum(axis=1)
inv_network_result2 = inv_network_result2.sort_values(["total_weight"], ascending=[False]).reset_index(drop=True)
inv_network_result2 = pd.merge(inv_network_result2, inv_tech_df)
inv_network_result2.to_excel('./last2/tech_connection/inv_network_result.xlsx', index=False, encoding='cp949')

In [109]:
inv_score2 = inv_network_result2.groupby('tech').mean()[['total_weight']]

In [110]:
inv_score2.sort_values('total_weight', ascending=False).to_excel('./last2/tech_connection/inv_tech.xlsx')

In [111]:
inv_score2.index.name=None

#### Total weight

In [112]:
total_weight_index2 = list(set(company_score2.index.tolist() + inv_score2.index.tolist() + tech_network_result2.node.tolist()))
total_score2 = pd.DataFrame(index=total_weight_index2)

In [113]:
company_score2.rename(columns={'total_weight':'NS'}, inplace=True)
tech_score2.rename(columns={'total_weight':'TS'}, inplace=True)
inv_score2.rename(columns={'total_weight':'IS'}, inplace=True)

In [114]:
company_score2.to_excel('./last2/tech_connection/NS.xlsx')
tech_score2.to_excel('./last2/tech_connection/TS.xlsx')
inv_score2.to_excel('./last2/tech_connection/IS.xlsx')

In [115]:
total_score2 = total_score2.join(company_score2)
total_score2 = total_score2.join(tech_score2)
total_score2 = total_score2.join(inv_score2)

In [116]:
total_score_df2 = pd.DataFrame(total_score2.fillna(0).sum(axis=1), columns=['Total score']).sort_values('Total score', ascending=False)

In [117]:
total_score_df2.to_excel('./last2/tech_connection/TOS.xlsx')